In [9]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 33.7 MB/s eta 0:00:00a 0:00:01


### for ref for package above
#### https://shimat.github.io/opencvsharp_docs/html/6121915d-1174-7345-bdca-789ee1373642.htm
#### https://www.geeksforgeeks.org/opencv-python-tutorial/?ref=shm 

### splitting clips up from the video

In [ ]:
import cv2
import os
# helper func to crop side borders from vid
# not implemented yet
def crop_borders(whatever,param,needed):
    # maybe use link below:
    # https://ffmpeg.org/ffmpeg-filters.html#cropdetect
    return None

# helper func to take in scene changes in the compilation vid
# output will be list of where scene changes happen 
# so we don't have to manually skim thru the vid
# and mark down where the clip starts and ends
def detect_scenes(video_path, threshold):
    # initialization
    cap = cv2.VideoCapture(video_path)
    scene_changes = []
    last_frame = None
    frame_index = 0

    #edge case/ handling if vid not there
    if not cap.isOpened():
        print("vid not present")
        return scene_changes

    while True:
        success, frame = cap.read()
        if not success:
            break

        # convert to BGR, less compute + easy processing pixels
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # compares pixel intensity btwn frames if last frame is present, retrieves diff in values
        if last_frame is not None:

            # FIX LATER
            # some clips are not getting indexed bc pixel intensity are similar
            # while some clips are getting indexed every frame bc too much of a difference
            diff = cv2.absdiff(gray_frame, last_frame)
            mean_diff = diff.mean()

            # checks if greater than threshold before recording a scene change
            if mean_diff > threshold:
                scene_changes.append(frame_index)

        # moves up vid
        last_frame = gray_frame
        frame_index += 1

    # avoid cv error
    cap.release()

    return scene_changes

# helper func to extract clips based on prev detect_scenes and crop_border (not implemented yet)
def extract_clips(video_path, scene_changes, output_dir):
    # initialization
    cap = cv2.VideoCapture(video_path)

    #edge case/ handling if vid not there
    if not cap.isOpened():
        print("vid not present")
        return

    # encode vid + retrieve dims
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
       
    vid_dimensions = (width, height)

    os.makedirs(output_dir, exist_ok=True)

    # append last frame to ensure the final clip is extracted bc detect_scene doesn't account for it
    scene_changes.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

    for i in range(len(scene_changes) - 1):
        # initialize start and end times based on detect_scenes)
        start_frame = scene_changes[i]
        end_frame = scene_changes[i + 1]

        output_path = os.path.join(output_dir, f"clip_{i + 1}.mp4")

        out = cv2.VideoWriter(output_path, fourcc, fps, vid_dimensions)

        #record
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        for frame_index in range(start_frame, end_frame):
            success, frame = cap.read()
            if not success:
                break

        out.release()
        print(f"done with clip {i}")

    cap.release()

if __name__ == "__main__":
    print("start")
    #threshold_vals = [20,30,40, 50,60, 70]
    #for val in threshold_vals:
    #    print(f"testing {val}")
    #    scene_changes = detect_scenes("data/manuallyedited.mp4", val)
    #    print(len(scene_changes))


    # need to start experimenting w/ threshold value
    scene_changes = detect_scenes("data/manuallyedited.mp4", 45)
    print(scene_changes)
    extract_clips("data/manuallyedited.mp4", scene_changes, "data/clips")

start
[355, 707, 1033, 1384, 1706, 2057, 2411, 2719, 3073, 3427, 3779, 4126, 4451, 4767, 5103, 5288, 5455, 5772, 6083, 6435, 6786, 7098, 7347, 7700, 7763, 7930, 7945, 7976, 7977, 7990, 7992, 7993, 7999, 8001, 8003, 8004, 8005, 8006, 8021, 8368, 8724, 9080, 9432]
done with clip 0
done with clip 1
done with clip 2
done with clip 3
done with clip 4
done with clip 5
done with clip 6
done with clip 7
done with clip 8
done with clip 9
done with clip 10
done with clip 11
done with clip 12
done with clip 13
done with clip 14
done with clip 15
done with clip 16
done with clip 17
done with clip 18
done with clip 19
done with clip 20
done with clip 21
done with clip 22
done with clip 23
done with clip 24
done with clip 25
done with clip 26
done with clip 27
done with clip 28
done with clip 29
done with clip 30
done with clip 31
done with clip 32
done with clip 33
done with clip 34
done with clip 35
done with clip 36
done with clip 37
done with clip 38
done with clip 39
done with clip 40
done with